In [ ]:
import os
import pandas as pd
import openai
from openai import AzureOpenAI
import re
import openai
import argparse

In [ ]:
import os
os.environ["no_proxy"] = "10.156.254.10"
openai.api_key = "dtnumds"
openai.api_base = "http://10.156.254.10:8000/v1"
os.environ["OPENAI_API_KEY"] = "dtnumds"
os.environ["OPENAI_API_BASE"] = "http://10.156.254.10:8000/v1"


## Question generation exploration

L'objectif de ce notebook est d'explorer de manière exhaustive les possibilités offertes par les LLMs pour créer un dataset d'évaluation de nos méthodes de RAG.

Docs :
- https://mlflow.org/docs/latest/llms/rag/notebooks/question-generation-retrieval-evaluation.html
- https://www.promptingguide.ai/applications/synthetic_rag


In [ ]:
df = pd.read_csv('../../../data/2023005993-attachment.pdf.csv', sep='|')

In [ ]:
def extract_question(text):
    try :
        question = text.split('?')[0] +" ?"
    except :
        question = ""
        reponse = ""
    return question
def extract_reponse(text):
    try :
        reponse = text.split('?')[1]
    except :
        reponse = ""
    return reponse
df['eval_question'] = df['eval_Q&A'].apply(extract_question)
df['eval_reponse'] = df['eval_Q&A'].apply(extract_reponse)

In [ ]:
df['n_words'] = df.text.apply(lambda x: len(x.split()))

In [ ]:
texts = df.text.to_list()

In [ ]:
client = AzureOpenAI( api_key="dtnumds",
                azure_endpoint="http://10.156.254.10:8000/v1",
                api_version = "2023-07-01-preview" )

In [ ]:
for model in client.models.list().data :
    print(model.id)

In [ ]:
model = "mixtral-instruct"

In [ ]:
prompt_mixtral_qa = """[INST]
Ta mission est de générer une couple de question/réponse en français à partir d'un extrait d'un document. Tu généreras une question spécifique à partir du paragraphe donné. 
Par exemple : 
- Quelle est l'information clé dans le paragraphe donné ?

La réponse doit utilisé autant d'informations que possible. 
Si vous n'êtes pas en mesure de répondre, donnez la réponse 'Je ne sais pas'.
La réponse doit être informative et comporter au plus 3 phrases.

Paragraphe : {content}

Génére un unique couple de question/réponse en français à partir du paragraphe ci dessus : 
[/INST]
"""

In [ ]:
prompt_mixtral_q = """<s> [INST]
Ta mission est de générer une question en français à partir d'un extrait d'un document. Tu généreras une question spécifique à partir du paragraphe donné. 
Par exemple : 
- Quelle est l'information clé dans le paragraphe donné ?
- Combien d'emplois ont été implantés dans les services relocalisés en 2022 ?
[/INST]
</s>
[INST]
Paragraphe :
{content}

Génère une question en français à partir du paragraphe ci-dessus :
[/INST]
"""

prompt_mixtral_a = """<s> [INST]
Tu es un assistant documentaire. Ta missions est de répondre à des questions à partir uniquement du contexte suivant. Si tu ne connais pas l'information, tu diras que tu ne connais pas la réponse. Ta réponse sera concise. Tu utiliseras uniquement l'information pertiente. 
Ta réponse devra obligatoirement être en français, c'est un point très important.
Voici un exemple de réponse :
- L'IA générative se base sur l'architecture transformer développée par Google.
[/INST]
</s>
[INST]
Contexte : 
{context_str}

Question :
{query_str}  

Répond à la question en te basant sur le contexte ci-dessus
[/INST]

"""

In [ ]:
prompt_llama = """
### System :
Your task is to generate question and answer in French based on document.
### User :
Please generate a question in French asking for the key information in the given paragraph.
Also answer the questions using the information in the given paragraph.
Please ask the specific question instead of the general question, like
'What is the key information in the given paragraph?'.
Please generate the answer using as much information as possible.
If you are unable to answer it, please generate the answer as 'I don't know.'
The answer should be informative and should be more than 3 sentences.

Paragraph: {content}

### Assistant :
"""

In [ ]:
texts = texts[:10]

In [ ]:
inputs_mixtral_q = [prompt_mixtral_q.format(content = t) for t in texts]
#inputs_llama = [prompt_llama.format(content = t) for t in texts]


In [ ]:
question = [q.text for q in client.completions.create(prompt=inputs_mixtral_q, model="mixtral-instruct", temperature=0.0, top_p=0.01, max_tokens=2048).choices]

In [ ]:
question

In [ ]:
inputs_mixtral_a = [prompt_mixtral_a.format(context_str = t, query_str=q) for t,q in zip(texts, question)]


In [ ]:
answer = [a.text for a in client.completions.create(prompt=inputs_mixtral_a, model="mixtral-instruct", temperature=0.0, max_tokens=4096).choices]

In [ ]:
answer

## évaluation qualité de la génération 

Objectif vérifier que le dataset possède une vraie diversité pour éviter d'évaluer sur des cas d'usages trop uniforme

In [ ]:
import matplotlib.pyplot as plt
questions = df["eval_question"].to_list()
question_len = pd.DataFrame([len(q) for q in questions], columns=["length"])
question_len.hist(bins=50)
plt.title("Histogram of Question Lengths")
plt.xlabel("Question Length")
plt.ylabel("Frequency")
plt.show()

In [ ]:
# Embedding
question_embeddings = client.embeddings.create(
            input = questions,
            model= "dgfip-e5-large"
        )


In [ ]:
embs = [e.embedding for e in question_embeddings.data]

In [ ]:
import umap

In [ ]:
reducer = umap.UMAP()


In [ ]:
umap_embedding = reducer.fit_transform(embs)

In [ ]:
plt.scatter(
    umap_embedding[:, 0],
    umap_embedding[:, 1],)
plt.gca().set_aspect('equal', 'datalim')
plt.title('UMAP projection of the LLM generate question dataset', fontsize=12);

In [ ]:
# Similarité moyenne entre chunk et question générée

chunk_embeddings = [e.embedding for e in client.embeddings.create(
            input = df.text.to_list(),
            model= "dgfip-e5-large"
        ).data]


In [ ]:
def cossim(x, y):
    return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

In [ ]:
from tqdm import tqdm
import numpy as np

In [ ]:
cosim_relevancy = []
for i in tqdm(range(len(chunk_embeddings))):
    cosim_relevancy.append(cossim(embs[i], chunk_embeddings[i]))


On peut remarquer que la génération automatique de question via Mixtral est plutôt bonne, elle conduite à une bonne diversité thématique et les documents sont proches de leur chunk respectifs, ce qui est plutôt un bon point 

## Complex question generation 

Objectif : Généner une 